In [ ]:
import os
import sys
sys.path.append(os.path.abspath('..'))
from config import *

!cp ../data/acorns.clean.csv .
!cp ../data/ft.fa .
!cp ../data/flower_genes.fa .

# ファイル処理

CSV などのファイルからデータを読み込むには、Pandas などのライブラリを用いると便利です。しかし、生物データを記録するファイル形式は CSV に限らず、例えば塩基配列データを記録した FASTA や GenBank、遺伝子注釈情報を記録した GFF などが存在します。このような多様なフォーマットに対応するためには、自分でファイルを開いて中身を確認し、必要な情報を抽出する力が求められます。

## 読み取り

Python では、`open` 関数を使ってファイルを開きます。処理が終わったタイミングで、明示的にファイルを閉じる必要があります。次は、どんぐりのデータセット（acorns.clean.csv）の最初の行を読み取り、ファイルを閉じる例です。なお、`readline` メソッドはファイルの内容を 1 行だけ読み込む関数です。

In [ ]:
# !wget https://py.biopapyrus.jp/data/acorns.clean.csv
fh = open('acorns.clean.csv')
print(fh.readline())
fh.close()

ファイルの内容をすべて出力したり、あるいは変数に保存したい場合は、for 構文を使います。`open` 関数で作られたオブジェクト `fh` は、見かけ上、ファイルの各行が要素となっているリストのように扱えます。そのため、`fh` を for 文で回せば、ファイルの中身を 1 行ずつ取り出すことができます。

In [ ]:
fh = open('acorns.clean.csv')

for x in fh:
    print(x)

fh.close()

このようにファイルを開いて、各行のデータを順番に取り出すことができるようになりました。次に、各行に記録されているどんぐりのデータセットのうち、重さだけを取り出してリストに格納する例を見ていきましょう。重さのデータは、ファイルの各行をカンマ区切りで分割したとき、2 番目の位置にある値です。そこで、`split` メソッドを利用して文字列を分割し、その 2 番目の値をリスト `w` に追加する（`w.append`）処理を書いていきます。

In [ ]:
w = []

fh = open('acorns.clean.csv')

for x in fh:
    d = x.split(',')
    w.append(d[1])

fh.close()

w

ただし、この処理ではファイル 1 行目のヘッダーもデータとしてリストに追加されてしまいます。そのため、ファイルを読み込むときに、1 行目だけスキップする処理（`continue`）を加える必要があります。

In [ ]:
w = []

fh = open('acorns.clean.csv')

i = 0
for x in fh:
    i = i + 1
    if i == 1:
        continue
    
    d = x.split(',')
    w.append(d[1])


fh.close()

w


また、こうして取り出した重さのデータは文字列のままになっています。そこで、リストに追加する際に `float` 関数を使って小数に変換する処理を加えます。

In [ ]:
w = []

fh = open('acorns.clean.csv')

i = 0
for x in fh:
    i = i + 1
    if i == 1:
        continue
    
    d = x.split(',')
    w.append(float(d[1]))


fh.close()

w


このように、テキストデータから必要な情報を取り出す基本的な方法は、まずファイルを開き、for文を使って各行を順に処理し、最後にファイルを閉じるという流れです。この手順を踏むことで、どのようなフォーマットのファイルであっても、効率的に必要な情報を抽出することが可能です。

なお、ファイルの閉じ忘れを防ぐため、Python では `with` 文を使ってファイルを読み込むことが推奨されています。この構文を使えば、処理の終了とともにファイルが自動的に閉じられます。ファイルの開き方が `with open() as fh` のように変わる点と、`fh.close()` が不要になる点を除けば、それ以外の処理は通常と同じです。

In [ ]:
w = []

with open('acorns.clean.csv') as fh:

    i = 0
    for x in fh:
        i = i + 1
        if i == 1:
            continue
        
        d = x.split(',')
        w.append(float(d[1]))

w


```{admonition} 練習問題 TF-1
どんぐりデータセット（`acorns.clean.csv`）を読み込み、クヌギのどんぐりに対応する重さ・高さ・直径のデータを、それぞれ変数 `k_weight`、`k_height`、`k_diameter` に格納するプログラムを作成せよ。
```


```{admonition} 練習問題 TF-2
どんぐりデータセット（`acorns.clean.csv`）を読み込み、樹種・重さ・高さ・直径のデータを抽出し、Pandas のデータフレームに変換するプログラムを作成せよ。ただし、Pandas ライブラリの `pd.read_csv` 関数などは使用しないこと。

```


```{admonition} 練習問題 TF-3
哺乳類睡眠データセット（`msleep.txt`）を読み込み、すべてのデータを Pandas のデータフレームに変換するプログラムを作成せよ。ただし、Pandas ライブラリの `pd.read_csv` 関数などは使用しないこと。このファイルにはコメント行が含まれており、これらはデータとして読み込まないように注意すること。また、欠損値は `NA` として記録されている。
```


FASTA ファイルについて見ていきましょう。FASTA ファイルは、塩基配列を記録するための形式です。最初の行は `>` 記号から始まり、この行には遺伝子名やそのほかの情報が記述されます。続く行には、その遺伝子の塩基配列が 1 行または複数行にわたって記述されます。基本的に、ファイルの末尾、または次に `>` が現れるまでの塩基配列は、ひとつの遺伝子に対応する情報です。以下は、FT 遺伝子の塩基配列が 3 行にわたって記述されている例です。

```
>FT
ACAAAAACAAGTAAAACAGAAACAATCAACACAGAGAAACCACCTGTTTGTTCAAGATCAAAGATGTCTA
TAAATATAAGAGACCCTCTTATAGTAAGCAGAGTTGTTGGAGACGTTCTTGATCCGTTTAATAGATCAAT
CACTCTAAAGGTTACTTATGGCCAAAGAGAGGTGACTAATGGCTTGGATCTAAGGCCTTCTCAGGTTCAA
```

では、実際のデータファイル（ft.fa）を使って内容を読み込んでみましょう。

In [ ]:
# !wget https://py.biopapyrus.jp/data/ft.fa
with open('ft.fa') as fh:
    for line in fh:
        print(line)

ファイルが正しく読み込めることを確認できたら、次に FT 遺伝子の塩基配列のうち、A、C、G、T の出現回数を調べてみましょう。`open` 関数で作成される `fh` オブジェクトには、ファイルの各行が含まれています。このうち、`>` から始まる行を除けば、それ以外の行はすべて塩基配列です。各行は文字列であり、1 文字ずつが塩基（A、C、G、T）に対応します。そのため、各行をループで回して 1 文字ずつ取り出し、出現回数をカウントすることができます。

In [ ]:
nA = 0
nC = 0
nG = 0
nT = 0

with open('ft.fa') as fh:
    for line in fh:
        for base in line:
            if base == 'A':
                nA = nA + 1
            elif base == 'C':
                nC = nC + 1
            elif base == 'G':
                nG = nG + 1
            elif base == 'T':
                nT = nT + 1

print(f'{nA=} {nC=} {nG=} {nT=}')

次に、同じデータを利用して、塩基配列の長さを求めてみましょう。上のプログラムでは、`base` には各塩基が代入されています。塩基が現れるたびに 1 を加算すれば、塩基配列の長さ（つまり遺伝子の長さ）を求めることができます。

In [ ]:
ft_length = 0

with open('ft.fa') as fh:
    for line in fh:
        for base in line:
            ft_length = ft_length + 1

ft_length

しかし、このプログラムには一つバグがあります。`for base in line` でループを回したとき、各行（`line`）の文字を一つずつ取り出して `base` に代入していますが、このとき、行末の改行コード（`\n`）も 1 文字として含まれてしまいます。

塩基の出現回数をカウントする際には、`base` が `'A'`、`'C'`、`'G'`、`'T'` と一致するかどうかを判定していたため、この改行コードは無視され、問題が表面化しませんでした。しかし、塩基配列の長さを単純に文字数として数えると、各行ごとに改行コード 1 文字分が余分にカウントされてしまいます。

この問題を回避するために、`rstrip()` メソッドを使って、各行の末尾にある改行コードなどの余分な文字を削除してから処理を行います。

In [ ]:
ft_length = 0

with open('ft.fa') as fh:
    for line in fh:
        line = line.rstrip()
        for base in line:
            ft_length = ft_length + 1

ft_length


```{admonition} 練習問題 TF-4
FT 遺伝子の塩基配列データ（ft.fa）を読み込み、塩基 A、C、G、T の出現確率（出現回数を塩基全長で割った値）を求め、グラフでわかりやすく示せ。
```


```{admonition} 練習問題 TF-5
開花関連遺伝子の塩基配列データ（flower_genes.fa）を読み込み、各遺伝子について、塩基 A、C、G、T の出現確率（出現回数を塩基全長で割った値）を求め、グラフでわかりやすく示せ。このファイルには複数の遺伝子が含まれていることに注意すること。
```


練習問題はスムーズに解けた。でも実務に踏み込んだ瞬間、「ヘッダー行が長すぎて読み込めない FASTA」、「GTF と言いつつ、なぜか一部が GFF」、「列数が微妙に違う行が混在し、毎回違う結果が出る解析」、「タブとスペースが混在していることに気づかず突き進む」など、背筋が凍る現象が次々に襲いかかってくる。それが現実。この業界、闇が深い。

## 書き込み

既存のデータをファイルに書き込む場合も、`open` 関数を使ってファイルを書き込みモードで開き、ファイルオブジェクト `outfh` を作成します。この `outfh` の `write` メソッドを用いてデータを書き込みます。たとえば、hello.txt というファイルを作成し、その中に "hello, world" という文字列を書き込む例を見てみましょう。　

In [ ]:
with open('hello.txt', mode = 'w') as outfh:
    outfh.write('hello, world')

このコードを実行すると、Jupyter Notebook を実行しているディレクトリと同じ場所に hello.txt ファイルが作成されます。実際にそのファイルを開いて中身を確認してみましょう。

ファイルに書き込む際、データは基本的に文字列である必要があります。数値などが含まれているとエラーになります。

In [ ]:
with open('hello.txt', mode = 'w') as outfh:
    outfh.write('hello, world')
    outfh.write(2025)

そのため、分析結果などをファイルに書き込む場合には、`str` 関数などを使って数値を文字列に変換する必要があります。さらに、for 文を使う際には、[`zip` 関数](python-zip)を利用することで、同じ長さのリストを複数同時にループ処理できます。

In [ ]:
trees = ['kunugi', 'kunugi', 'arakshi', 'shirakashi']
weights = [5.55, 5.23, 1.42, 1.64]

with open('donguri.txt', mode = 'w') as outfh:
    outfh.write('tree,weight')
    for tree, weight in zip(trees, weights):
        outfh.write(f'{tree},{weight}')

しかし、このようにして書き込まれたファイルは、すべてのデータが 1 行に連続して書き込まれてしまいます。これは、`write` 関数が「1 回呼び出すごとに 1 行を書く」関数ではなく、渡された文字列をそのままファイルに書き込むだけの関数だからです。つまり、書き込むデータの中に改行コード（`\n`）が含まれていなければ、すべてのデータが同じ行に並んでしまいます。逆に、データの中に改行コードを含めれば、含まれた分だけ改行されることになります。

In [ ]:
trees = ['kunugi', 'kunugi', 'arakshi', 'shirakashi']
weights = [5.55, 5.23, 1.42, 1.64]

with open('donguri.txt', mode = 'w') as outfh:
    outfh.write('tree,weight\n')
    for tree, weight in zip(trees, weights):
        outfh.write(f'{tree},{weight}\n')

In [ ]:
!rm hello.txt
!rm donguri.txt
!rm acorns.clean.csv
!rm ft.fa
!rm flower_genes.fa